In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [3]:
from graphviz import Digraph

def trace(root):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges


def draw_dot(root):
    dot = Digraph(format="png", graph_attr={"rankdir":"LR"})  # LR = left to right

    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name=uid, label="{%s | data %.4f | grad %.4f}" % (n.label, n.data, n.grad), shape='record')
        if n._operation:
            # if this value is a result of some operation, create an op node for it
            dot.node(name=uid + n._operation, label=n._operation)
            # and connect this node to it
            dot.edge(uid + n._operation, uid)

    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._operation)

    return dot

In [22]:
class Value:
    def __init__(self, data, _children = (), _operation = "", label = ""):
        self.data = data
        self._prev = set(_children)
        self._operation = _operation
        self._label = label
        
    def __repr__(self):
        return f"Value(data = {self.data})"
    
    def __add__(self, other):
        addition = self.data + other.data
        operation = " + "
        children = (self, other)
        result = Value(data = addition, _children=children, _operation=operation)
        return result
    
    def __mul__(self, other):
        multiplication = self.data * other.data
        operation = " x "
        children = (self, other)
        result = Value(data = multiplication, _children=children, _operation=operation)
        return result
    
    

In [26]:
a = Value(2.0, "a")
b = Value(3.0, "b")

In [27]:
c = a + b; c.label = "c"
d = a * b; d.label = "d"

In [28]:
c._operation

' + '

In [29]:
d._operation

' x '

In [30]:
d._prev

{Value(data = 2.0), Value(data = 3.0)}

In [31]:
c._prev

{Value(data = 2.0), Value(data = 3.0)}